# File Info

- A hodgepodge of data exploration, one-off file processing (converting spider .sql files to .db files for LangChain), and whatever else I needed a disorganized notebook for 

In [3]:
from datasets import load_dataset
import pandas as pd
import os
from langchain_community.utilities.sql_database import SQLDatabase



/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Data

In [4]:
dataset = load_dataset("xlangai/spider")


In [29]:
dataset['validation']

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})

In [3]:
dataset['train']

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})

In [14]:
df_train = dataset['train'].to_pandas()

df_test = dataset['validation'].to_pandas()

In [25]:
df_test['db_id'].unique()
result = df_test[df_test['db_id'] == 'car_1']
x = result['question']

In [28]:
x.to_list()

['How many continents are there?',
 'What is the number of continents?',
 'How many countries does each continent have? List the continent id, continent name and the number of countries.',
 'For each continent, list its id, name, and how many countries it has?',
 'How many countries are listed?',
 'How many countries exist?',
 'How many models does each car maker produce? List maker full name, id and the number.',
 'What is the full name of each car maker, along with its id and how many models it produces?',
 'Which model of the car has the minimum horsepower?',
 'What is the model of the car with the smallest amount of horsepower?',
 'Find the model of the car whose weight is below the average weight.',
 'What is the model for the car with a weight smaller than the average?',
 'Find the name of the makers that produced some cars in the year of 1970?',
 'What is the name of the different car makers who produced a car in 1970?',
 'Find the make and production time of the cars that were 

In [20]:
car_Qs = []
for idx, row in df_train.iterrows():
    if row['db_id'] == 'country_language':
        car_Qs.append(row['question'])



In [21]:
car_Qs

[]

In [6]:
result = df_train.loc[df_train['db_id'].str.contains('chinook', na=False)]
result

,db_id,query,question,query_toks,query_toks_no_value,question_toks
807,chinook_1,SELECT count(*) FROM ALBUM,How many albums are there?,"[SELECT, count, (, *, ), FROM, ALBUM]","[select, count, (, *, ), from, album]","[How, many, albums, are, there, ?]"
808,chinook_1,SELECT count(*) FROM ALBUM,Find the number of albums.,"[SELECT, count, (, *, ), FROM, ALBUM]","[select, count, (, *, ), from, album]","[Find, the, number, of, albums, .]"
809,chinook_1,SELECT Name FROM GENRE,List the names of all music genres.,"[SELECT, Name, FROM, GENRE]","[select, name, from, genre]","[List, the, names, of, all, music, genres, .]"
810,chinook_1,SELECT Name FROM GENRE,What are the names of different music genres?,"[SELECT, Name, FROM, GENRE]","[select, name, from, genre]","[What, are, the, names, of, different, music, ..."
811,chinook_1,"SELECT * FROM CUSTOMER WHERE State = ""NY""",Find all the customer information in state NY.,"[SELECT, *, FROM, CUSTOMER, WHERE, State, =, `...","[select, *, from, customer, where, state, =, v...","[Find, all, the, customer, information, in, st..."
...,...,...,...,...,...,...
886,chinook_1,SELECT distinct(UnitPrice) FROM TRACK,Find the distinct unit prices for tracks.,"[SELECT, distinct, (, UnitPrice, ), FROM, TRACK]","[select, distinct, (, unitprice, ), from, track]","[Find, the, distinct, unit, prices, for, track..."
887,chinook_1,SELECT count(*) FROM ARTIST WHERE artistid NOT...,How many artists do not have any album?,"[SELECT, count, (, *, ), FROM, ARTIST, WHERE, ...","[select, count, (, *, ), from, artist, where, ...","[How, many, artists, do, not, have, any, album..."
888,chinook_1,SELECT count(*) FROM ARTIST WHERE artistid NOT...,Cound the number of artists who have not relea...,"[SELECT, count, (, *, ), FROM, ARTIST, WHERE, ...","[select, count, (, *, ), from, artist, where, ...","[Cound, the, number, of, artists, who, have, n..."
889,chinook_1,SELECT T1.Title FROM Album AS T1 JOIN Track AS...,What are the album titles for albums containin...,"[SELECT, T1.Title, FROM, Album, AS, T1, JOIN, ...","[select, t1, ., title, from, album, as, t1, jo...","[What, are, the, album, titles, for, albums, c..."


In [7]:
X_test_query_chinook = result['question']
y_test_query_chinook = []

for index, row in result['query'].iloc[0:100].iteritems():
    y_test_query_chinook.append((row, result['db_id'][index]))



In [8]:
y_test_query_chinook

[('SELECT count(*) FROM ALBUM', 'chinook_1'),
 ('SELECT count(*) FROM ALBUM', 'chinook_1'),
 ('SELECT Name FROM GENRE', 'chinook_1'),
 ('SELECT Name FROM GENRE', 'chinook_1'),
 ('SELECT * FROM CUSTOMER WHERE State  =  "NY"', 'chinook_1'),
 ('SELECT * FROM CUSTOMER WHERE State  =  "NY"', 'chinook_1'),
 ('SELECT FirstName ,  LastName FROM EMPLOYEE WHERE City  =  "Calgary"',
  'chinook_1'),
 ('SELECT FirstName ,  LastName FROM EMPLOYEE WHERE City  =  "Calgary"',
  'chinook_1'),
 ('SELECT distinct(BillingCountry) FROM INVOICE', 'chinook_1'),
 ('SELECT distinct(BillingCountry) FROM INVOICE', 'chinook_1'),
 ('SELECT Name FROM ARTIST WHERE Name LIKE "%a%"', 'chinook_1'),
 ('SELECT Name FROM ARTIST WHERE Name LIKE "%a%"', 'chinook_1'),
 ('SELECT Title FROM ALBUM AS T1 JOIN ARTIST AS T2 ON T1.ArtistId  =  T2.ArtistId WHERE T2.Name  =  "AC/DC"',
  'chinook_1'),
 ('SELECT Title FROM ALBUM AS T1 JOIN ARTIST AS T2 ON T1.ArtistId  =  T2.ArtistId WHERE T2.Name  =  "AC/DC"',
  'chinook_1'),
 ('SELECT 

In [9]:
X_test_query_chinook

807                           How many albums are there?
808                           Find the number of albums.
809                  List the names of all music genres.
810        What are the names of different music genres?
811       Find all the customer information in state NY.
                             ...                        
886            Find the distinct unit prices for tracks.
887              How many artists do not have any album?
888    Cound the number of artists who have not relea...
889    What are the album titles for albums containin...
890    Find the titles of albums that contain tracks ...
Name: question, Length: 84, dtype: object

### 100 ROWS

In [10]:
X_test_query = (df_test['question'][0:100],df_test['db_id'])
y_test_query = (df_test['query'][0:100], df_test['db_id'])

In [11]:
X_test_query = []
y_test_query = []

for index, row in df_test['question'].iloc[0:100].iteritems():
    X_test_query.append(row)

for index, row in df_test['query'].iloc[0:100].iteritems():
    y_test_query.append((row, df_test['db_id'][index]))


In [12]:
y_test_query

[('SELECT count(*) FROM singer', 'concert_singer'),
 ('SELECT count(*) FROM singer', 'concert_singer'),
 ('SELECT name ,  country ,  age FROM singer ORDER BY age DESC',
  'concert_singer'),
 ('SELECT name ,  country ,  age FROM singer ORDER BY age DESC',
  'concert_singer'),
 ("SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'",
  'concert_singer'),
 ("SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'",
  'concert_singer'),
 ('SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1',
  'concert_singer'),
 ('SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1',
  'concert_singer'),
 ('SELECT DISTINCT country FROM singer WHERE age  >  20', 'concert_singer'),
 ('SELECT DISTINCT country FROM singer WHERE age  >  20', 'concert_singer'),
 ('SELECT country ,  count(*) FROM singer GROUP BY country', 'concert_singer'),
 ('SELECT country ,  count(*) FROM singer GROUP BY country', 'concert_singer'),
 ('SEL

### Write to .txt files:

#### Gold

In [13]:
with open('gold.txt', 'w') as file:

    for element in y_test_query_chinook:
        file.write(element[0] + '\t' + element[1] + '\n')


#### Predictions

In [14]:
# with open('pred.txt', 'w') as file:

#     # for element in :
#     #     file.write(element[0] + '\t' + element[1] + '\n')

### Create .db files

In [15]:
import sqlite3
import pos

def sql_to_db(sql_file, new_db_filename):
    conn = sqlite3.connect('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/' + new_db_filename + '.db')

    # Read and execute the schema.sql file
    with open(sql_file, 'r') as f:
        conn.executescript(f.read())

    conn.commit()
    conn.close()

# sql_to_db('/Users/mymac/LLM/Personal-Medical-Assistant/backend/testing/spider/test_database/academic/schema.sql','academic_test')


In [16]:
s = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/test_database/browser_web/schema.sql'

s[84:-11]  

'browser_web'

In [18]:

with open('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/sql_files.txt', 'r') as file:
    # Read each line in the file
    lines = file.readlines()
    start_int = 81
    end_int = -12
    
    for line in lines:

        # try:

            line = line.strip()
            new_filename = line[84:-11]              #naively index into the name of the database given absolute path
            sql_to_db(line,new_db_filename=new_filename)

        # except:
        #     print(f"error occurred: {new_filename}")


OperationalError: unable to open database file

In [23]:
def get_schema(_):
    return DB.get_table_info()

In [24]:
directory_path = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas'
schemas = []
count = 0
# Walk through the directory
for root, dirs, files in os.walk(directory_path):
    for file in files:
        count += 1

        try:
            DB = SQLDatabase.from_uri("sqlite:///" + directory_path + '/' + file)
            schemas.append(get_schema(''))

        except:
            print("Error occurred: " + file)

/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(
/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  self._metadata.reflect(


Error occurred: store_product.db
Error occurred: book_1.db
Error occurred: flight_1.db
Error occurred: music_1.db


In [26]:
len(schemas)

37

In [ ]:
directory_path = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/test_database'

# Output file to write the paths
output_file = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/sql_files.txt'

with open(output_file, 'w') as outfile:
    for root, dirs, files in os.walk(directory_path):
        # List subdirectories directly within the current directory
        if root == os.path.join(directory_path):
            for directory in dirs:
                print("DIR", directory)

                directory_files = os.listdir(os.path.join(directory_path, directory))
                print("__Files in", directory)
                for file in directory_files:
                    if file.endswith('.sql'):
                        file_path = os.path.join(root, directory, file)
                        print(file)
                        outfile.write(file_path + '\n')

DIR browser_web
__Files in browser_web
schema.sql
DIR musical
__Files in musical
schema.sql
DIR country_language
__Files in country_language
schema.sql
DIR farm
__Files in farm
schema.sql
DIR voter_1
__Files in voter_1
DIR game_injury
__Files in game_injury
schema.sql
DIR hospital_1
__Files in hospital_1
schema.sql
DIR manufacturer
__Files in manufacturer
schema.sql
DIR station_weather
__Files in station_weather
schema.sql
DIR perpetrator
__Files in perpetrator
schema.sql
DIR soccer_3
__Files in soccer_3
schema.sql
DIR storm_record
__Files in storm_record
schema.sql
DIR flight_1
__Files in flight_1
schema.sql
DIR manufactory_1
__Files in manufactory_1
schema.sql
DIR cre_Theme_park
__Files in cre_Theme_park
schema.sql
DIR museum_visit
__Files in museum_visit
schema.sql
DIR race_track
__Files in race_track
schema.sql
DIR soccer_2
__Files in soccer_2
schema.sql
DIR bike_1
__Files in bike_1
schema.sql
DIR bbc_channels
__Files in bbc_channels
schema.sql
DIR pilot_record
__Files in pilot_rec

In [ ]:
# Iterate over all files in the directory
import os

directory_path = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/test_database'

paths = []

for root, dirs, files in os.walk(directory_path):
    if root == os.path.join(directory_path):
        # print("Subdirectories directly within the current directory:")
        for directory in dirs:
            print("DIR",directory)

            directory_files = os.listdir(os.path.join(directory_path, directory))
            print("__Files in", directory)
            for file in directory_files:
                # print(file)
                # try:

                if file.endswith('.sql'):
                    print(file)


                    
                #     new_filename = file[:-4]
                #     sql_to_db(file, directory)


with open('sql_files.txt', 'w') as file:

    for p in paths:
        file.write('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable' + )

DIR browser_web
__Files in browser_web
schema.sql
DIR musical
__Files in musical
schema.sql
DIR country_language
__Files in country_language
schema.sql
DIR farm
__Files in farm
schema.sql
DIR voter_1
__Files in voter_1
DIR game_injury
__Files in game_injury
schema.sql
DIR hospital_1
__Files in hospital_1
schema.sql
DIR manufacturer
__Files in manufacturer
schema.sql
DIR station_weather
__Files in station_weather
schema.sql
DIR perpetrator
__Files in perpetrator
schema.sql
DIR soccer_3
__Files in soccer_3
schema.sql
DIR storm_record
__Files in storm_record
schema.sql
DIR flight_1
__Files in flight_1
schema.sql
DIR manufactory_1
__Files in manufactory_1
schema.sql
DIR cre_Theme_park
__Files in cre_Theme_park
schema.sql
DIR museum_visit
__Files in museum_visit
schema.sql
DIR race_track
__Files in race_track
schema.sql
DIR soccer_2
__Files in soccer_2
schema.sql
DIR bike_1
__Files in bike_1
schema.sql
DIR bbc_channels
__Files in bbc_channels
schema.sql
DIR pilot_record
__Files in pilot_rec

In [ ]:
df_train

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."
...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,What are all the company names that have a boo...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...","[What, are, all, the, company, names, that, ha..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",Show the movie titles and book titles for all ...,"[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...","[Show, the, movie, titles, and, book, titles, ..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",What are the titles of movies and books corres...,"[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...","[What, are, the, titles, of, movies, and, book..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,Show all company names with a movie directed i...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...","[Show, all, company, names, with, a, movie, di..."


In [1]:
from langchain_community.llms import Ollama

LLM = Ollama(model="phi3")      #choose LLM


Ollama
Params: {'model': 'phi3',
